In [4]:
import requests
from collections import defaultdict

# Replace these with the URLs of your "ventas" and "productos" API endpoints
ventas_api_url = "http://localhost:3001/ventas"
productos_api_url = "http://localhost:3001/productos"

# Retrieve ventas data
ventas_response = requests.get(ventas_api_url)

if ventas_response.status_code == 200:
    ventas_data = ventas_response.json()[0]  # Access the first element, which is the list of sales

    product_quantities = defaultdict(int)

    # Calculate product quantities sold
    for venta in ventas_data:
        product_name = venta["nombre_producto"]
        quantity = int(venta["cantidad"])
        product_quantities[product_name] += quantity

    # Filter products with stock above 0
    above_0_stock = [product for product, quantity in product_quantities.items() if quantity > 0]

    # Retrieve productos data
    productos_response = requests.get(productos_api_url)

    if productos_response.status_code == 200:
        product_data = productos_response.json()[0]  # Access the first element, which is the list of products

        # Create separate lists for each category
        above_0_stock_list = [product for product in product_data if product["nombre_producto"] in above_0_stock]
        below_10_stock_list = [product for product in product_data if int(product.get("stock", 0)) < 10]

        # Identify products with the absolute lowest stock
        min_stock = min(product_data, key=lambda product: int(product.get("stock", float('inf'))))
        lowest_stock_list = [product for product in product_data if int(product.get("stock", 0)) == int(min_stock.get("stock", 0))]

        # Display products in the desired order
        print("\nProducts with stock above 0:")
        for product in above_0_stock_list:
            print(f"{product['nombre_producto']} - Quantity Sold: {product_quantities[product['nombre_producto']]}")

        print("\nProducts with stock below 10:")
        for product in below_10_stock_list:
            print(f"{product['nombre_producto']} - Stock: {int(product.get('stock', 0))}")

        print("\nProduct(s) with the lowest stock:")
        for product in lowest_stock_list:
            print(f"{product['nombre_producto']} - Stock: {int(product.get('stock', 0))}")

    else:
        print("Failed to retrieve data from the 'productos' API")
else:
    print("Failed to retrieve data from the 'ventas' API")



Products with stock above 0:
Filtro de aceite Honda Premium - Quantity Sold: 3
Escape Suzuki Carbono Racing - Quantity Sold: 5
Guantes Suzuki Leather Pro - Quantity Sold: 2
Batería Yamaha Lithium Performance - Quantity Sold: 3
Chaqueta Yamaha Textile Adventure - Quantity Sold: 1
Chaqueta Ducati Racing Leather - Quantity Sold: 2
Kit de mantenimiento Ducati Multistrada - Quantity Sold: 4
Casco Bajaj Street Rider - Quantity Sold: 1
Guantes de cuero Bajaj Pro - Quantity Sold: 2

Products with stock below 10:

Product(s) with the lowest stock:
Kit de mantenimiento Ducati Multistrada - Stock: 25


In [15]:
import requests
import pandas as pd
from collections import defaultdict

# Resto del código...

if ventas_response.status_code == 200:
    ventas_data = ventas_response.json()[0]  # Acceder al primer elemento, que es la lista de ventas

    product_quantities = defaultdict(int)

    # Cálculo de las cantidades de productos vendidos
    for venta in ventas_data:
        product_name = venta["nombre_producto"]
        quantity = int(venta["cantidad"])
        product_quantities[product_name] += quantity

    # Filtrar productos con stock mayor a 0
    above_0_stock = [product for product, quantity in product_quantities.items() if quantity > 0]

    # Resto del código...

    # Mostrar productos en el orden deseado
    above_0_stock_data = []
    below_10_stock_data = []
    lowest_stock_data = []

    for product in above_0_stock_list:
        above_0_stock_data.append({
            'Producto': product['nombre_producto'],
            'Vendidos': product_quantities[product['nombre_producto']]
        })

    for product in below_10_stock_list:
        below_10_stock_data.append({
            'Productos': product['nombre_producto'],
            'Stock': int(product.get('stock', 0))
        })

    for product in lowest_stock_list:
        lowest_stock_data.append({
            'Productos': product['nombre_producto'],
            'Stock': int(product.get('stock', 0))
        })

    # Convertir a DataFrames de pandas
    above_0_stock_df = pd.DataFrame(above_0_stock_data)
    below_10_stock_df = pd.DataFrame(below_10_stock_data)
    lowest_stock_df = pd.DataFrame(lowest_stock_data)

    # Escribir a un archivo Excel
    with pd.ExcelWriter('productos_info.xlsx') as writer:
        above_0_stock_df.to_excel(writer, sheet_name='Con stock', index=False)
        below_10_stock_df.to_excel(writer, sheet_name='Con stock menor a 10', index=False)
        lowest_stock_df.to_excel(writer, sheet_name='Stock Bajo', index=False)
else:
    print("Falló al obtener datos de la API 'ventas'")
